## 1)Importing the required libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
import keras
from sklearn.utils import resample
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image
from sklearn.metrics import precision_recall_fscore_support, accuracy_score,confusion_matrix

## 2) Importing the dataset

In [2]:
n_iterations = 1000
training_data_gen = ImageDataGenerator()
traindata = training_data_gen.flow_from_directory(directory="/kaggle/input/surface-crack/train",target_size=(224,224))
validation_data_gen = ImageDataGenerator()
validdata = validation_data_gen.flow_from_directory(directory="/kaggle/input/surface-crack/valid", target_size=(224,224))

Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


## 3)Defining the functions used

label(x) a function which is used to create the list yreal, which has true class values of testset images

pred(x) , a function which is used to determine class of the image .It takes in the output of last dense layer of our model and returns the class it belongs to.

test(L,path,images), given a list L and path under which the images are present, will append the predicted classes of those images to L

In [3]:
def label(x):
    if(x<100):
        #First 100 are crack-negative in test set so Yreal is 0
        return 0
    else:
        #Next 100 are crack-positive in test set so Yreal is 1
        return 1
def pred(x):
    if(x[0]>=x[1]):
        return 0
    else:
        return 1
def test(L,path):
    images=os.listdir(path)
    for imagnames in images:
        img_path = path+"/"+imagnames
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_batch = np.expand_dims(img_array, axis=0)
        img_preprocessed = preprocess_input(img_batch)
        prediction = model.predict(img_preprocessed)
        L.append(pred(prediction[0]))

## 4)Defining the model

Model structure:

Inputimage(224 x 224 pixels rgb image)->VGG16->Dense(128 nodes)->Dense_output(2 nodes)

In [4]:
vgg = VGG16(weights='imagenet', include_top=True)
X= vgg.layers[-2].output
X = Dense(128, activation='relu')(X)
predictions = Dense(2, activation="softmax")(X)
model = Model(inputs = vgg.input, outputs = predictions)
model=Model(inputs=vgg.input, outputs = predictions)
checkpointname="vgg16_final.h5"
checkpoint = ModelCheckpoint(checkpointname, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')

2023-01-19 15:36:41.687709: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 15:36:41.688743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 15:36:41.867743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 15:36:41.868694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 15:36:41.869565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

553476096/553467096 [==============================] - 3s 0us/step


## 5)Training the model

We are training the model at a learning rate of 1e-4 over 100 epochs each iteration. we are training using training data , validating using validation data. The model gets saved after every iteration and predicts for the testset. when a testset accuracy of over 97% is achieved we stop and finalize our model.

In [ ]:
yreal=[label(x) for x in range(0,200)]
negpath="/kaggle/input/surface-crack/test/Negative"
pospath="/kaggle/input/surface-crack/test/Positive"
for i in range(n_iterations):
    ypred=[]
    print("\n\nIteration number: ",i,"\n\n")
    model.compile(loss = "categorical_crossentropy", optimizer = SGD(lr=0.0001), metrics=["accuracy"])
    model.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 100, validation_data= validdata, validation_steps=1, callbacks=[checkpoint,early])
    model.save_weights(checkpointname)
    test(ypred,negpath)
    test(ypred,pospath)
    precision, recall, f1_score, _ = precision_recall_fscore_support(yreal, ypred)
    avg_pre=(precision[0]+precision[1])/2
    acc = accuracy_score(yreal, ypred)
    print("\n Testing Accuracy is: ",acc)
    if(acc>0.97):
        cm=confusion_matrix(yreal,ypred)
        print("Model finalised.\nAccuracy: ",acc)
        print("Precision: ",precision)
        print("Recall: ",recall)
        print("F1: ",f1_score)
        print("Confusion matrix: ",cm)
        break



Iteration number:  0 




/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2023-01-19 15:36:51.386557: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2023-01-19 15:36:54.156298: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


2/2 [==============================] - 25s 1s/step - loss: 0.8497 - accuracy: 0.5312 - val_loss: 0.6508 - val_accuracy: 0.5938
Epoch 2/100
2/2 [==============================] - 2s 948ms/step - loss: 0.7445 - accuracy: 0.5625 - val_loss: 0.5029 - val_accuracy: 0.7812
Epoch 3/100
2/2 [==============================] - 1s 780ms/step - loss: 0.5758 - accuracy: 0.6875 - val_loss: 0.4881 - val_accuracy: 0.7500
Epoch 4/100
2/2 [==============================] - 1s 773ms/step - loss: 0.5544 - accuracy: 0.7188 - val_loss: 0.5081 - val_accuracy: 0.7500
Epoch 5/100
2/2 [==============================] - 1s 682ms/step - loss: 0.4821 - accuracy: 0.7812 - val_loss: 0.4469 - val_accuracy: 0.7812
Epoch 6/100
2/2 [==============================] - 1s 708ms/step - loss: 0.5143 - accuracy: 0.7812 - val_loss: 0.3969 - val_accuracy: 0.8750
Epoch 7/100
2/2 [==============================] - 1s 670ms/step - loss: 0.4127 - accuracy: 0.8438 - val_loss: 0.3533 - val_accuracy: 0.8438
Epoch 8/100
2/2 [==========

## 6)Predictions

In [ ]:
predictpath="/kaggle/input/surface-crack/predict"
predict_images=os.listdir(predictpath)
predictions=[]
test(predictions,predictpath)
length=len(predictions)
for i in range(0,length):
    statement=""
    if(predictions[i]==1):
        statement=" Crack is present."
    else:
        statement=" No sign of cracks."
    print(predict_images[i]+statement)